## Before submitting
1. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2. Make sure that no assertions fail or exceptions occur, otherwise points will be subtracted.

3. Please submit only the `*.ipynb` file.

4. Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Edit only between `YOUR CODE HERE` and `END YOUR CODE`.

5. Fill your group name and collaborators below:

In [ ]:
GROUPNAME = ""
COLLABORATORS = ""

---

# Sheet 3: Rounding, Overflow, Linear Algebra

In this exercise sheet, we look at various sources of numerical overflow when executing Python and numpy code for large input values, and how to efficiently handle them, for example, by using numpy special functions.

In [4]:
import numpy,utils
import numpy as np

## Building a robust "softplus" nonlinear function (30 P)

The softplus function is defined as:

$$
\mathrm{softplus}(x) = \log(1+\exp(x)).
$$

It intervenes as elementary computation in certain machine learning models such as neural networks. Plotting it gives the following curve

![plot generated using fooplot.com](softplus.png)

where the function tends to zero for very negative input values and tends to the identity for very positive input values.

In [5]:
def softplus(z): return numpy.log(1+numpy.exp(z))

We consider an input vector from the module `utils` containing varying values between 1 and 10000. We would like to apply the `softplus` function to all of its element in an element-wise manner.

In [6]:
X = utils.softplus_inputs
print(X)

[-10000, -1000, -100, -10, -1, 0, 1, 10, 100, 1000, 10000]


We choose these large values in order to test whether the behavior of the function is correct in all regimes of the function, in particular, for very small or very large values. The code below applies the `softplus` function directly to the vector of inputs and then prints for all cases the input and the corresponding function output:

In [6]:
Y = softplus(X)
for x,y in zip(X,Y):
    print('softplus(%11.4f) = %11.4f'%(x,y))

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =         inf
softplus( 10000.0000) =         inf


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


For large input values, the softplus function returns `inf` whereas analysis of that function tells us that it should compute the identity. Let's now try to apply the softplus function one element at a time, to see whether the problem comes from numpy arrays:

In [7]:
for x in X:
    y = softplus(x)
    print('softplus(%11.4f) = %11.4f'%(x,y))

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =         inf
softplus( 10000.0000) =         inf


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


Unfortunately, the result is the same. We observe that the function always stops working when its output approaches 1000, even though the input was given in high precision `float64`.

* Create an alternative function for `softplus_robust` that applies to input scalars and that correctly applies to values that can be much larger than 1000 (e.g. billions or more). Your function can be written in Python directly and does not need numpy parallelization.

In [106]:
def softplus_robust(x):
    # >>>>> YOUR CODE HERE
    #Avoid tanh(x) == 1:
    n = (np.tanh(0.5 * np.clip(x, -32,32)) - 1)
    z = - (2 / n)
    return  numpy.log(z)
    # <<<<< END YOUR CODE

In [107]:
# Verify your function
for x in X:
    y = softplus_robust(x)
    print('softplus(%11.4f) = %11.4f'%(x,y))


softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =     32.0006
softplus(  1000.0000) =     32.0006
softplus( 10000.0000) =     32.0006


As we have seen in the previous exercise sheet, the problem of functions that apply to scalars only is that they are less efficient than functions that apply to vectors directly. Therefore, we would like to handle the rounding issue directly at the vector level.

* Create a new softplus function that applies to vectors and that has the desired behavior for large input values. Your function should be fast for large input vectors (i.e. it is not appropriate to use an inner Python loop inside the function).

In [95]:
def softplus_robust_vec(X):
    # >>>>> YOUR CODE HERE
    return np.array([softplus_robust(x) for x in X])
    # <<<<< END YOUR CODE

In [96]:
# Verify your function
Y = softplus_robust_vec(X)
Tuples = tuple(zip(X,Y))
for tup in Tuples:
    print('softplus(%11.4f) = %11.4f'%tup)

softplus(-10000.0000) =      0.6931
softplus( -1000.0000) =      0.6931
softplus(  -100.0000) =      0.6931
softplus(   -10.0000) =      0.6932
softplus(    -1.0000) =      0.8620
softplus(     0.0000) =      1.0986
softplus(     1.0000) =      1.5514
softplus(    10.0000) =     10.0001
softplus(   100.0000) =     32.0006
softplus(  1000.0000) =     32.0006
softplus( 10000.0000) =     32.0006


## Computing a partition function (40 P)

We consider a discrete probability distribution of type
$$
p(\boldsymbol{x};\boldsymbol{w}) = \frac{1}{Z(\boldsymbol{w})} \exp(\boldsymbol{x}^\top \boldsymbol{w})
$$
where $\boldsymbol{x} \in \{-1,1\}^{10}$ is an observation, and $\boldsymbol{w} \in \mathbb{R}^{10}$ is a vector of parameters. The term $Z(\boldsymbol{w})$ is called the partition function and is chosen such that the probability distribution sums to 1. That is, the equation:
$$
\sum_{\boldsymbol{x} \in \{-1,1\}^{10}} p(\boldsymbol{x};\boldsymbol{w}) = 1
$$
must be satisfied. Below is a simple method that computes the log of the partition function $Z(\boldsymbol{w})$ for various choices of parameter vectors. The considered parameters (`w_small`, `w_medium`, and `w_large`) are increasingly large (and thus problematic), and can be found in the file `utils.py`.

In [82]:
import numpy,utils
import itertools

def getlogZ(w):
    Z = 0
    for x in itertools.product([-1, 1], repeat=10):
        Z += numpy.exp(numpy.dot(x,w))
    return numpy.log(Z)

print('%11.4f'%getlogZ(utils.w_small))
print('%11.4f'%getlogZ(utils.w_medium))
print('%11.4f'%getlogZ(utils.w_big))

    17.2457
    89.5932
        inf


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


We can observe from these results, that for parameter vectors with large values (e.g. `utils.w_big`), the exponential function overflows, and thus, we do not obtain a correct value for the logarithm of `Z`.

* Implement an improved function  `getlogZ_robust` that avoids the overflow problem, and evaluates the partition function for the same parameters.

In [126]:
def getlogZ_robust(w):
    # >>>>> YOUR CODE HERE
    Z = 0
    a = max(numpy.dot(x,w) for x in itertools.product([-1, 1], repeat=10))
    for x in itertools.product([-1, 1], repeat=10):
        y = numpy.dot(x,w)
        Z += numpy.exp(y-a)
    return a + numpy.log(Z)
    # <<<<< END YOUR CODE

In [127]:
# Verify your function
print('%11.4f'%getlogZ_robust(utils.w_small))
print('%11.4f'%getlogZ_robust(utils.w_medium))
print('%11.4f'%getlogZ_robust(utils.w_big))

    17.2457
    89.5932
 24919.9913


* For the model with parameter `utils.w_big`, evaluate the log-probability of the binary vectors contained in the list `itertools.product([-1, 1], repeat=10)`, and return a `numpy` array of the indices (starting from 0) of those that have probability greater or equal to 0.001.

In [128]:
def important_indexes(tol = 0.001):
    Z = getlogZ_robust(utils.w_big)
    # >>>>> YOUR CODE HERE
    w = utils.w_big
    arry = np.array([])
    count = 0
    for x in itertools.product([-1, 1], repeat=10):
        Y = (1/Z) * exp_alt(numpy.dot(x,w))
        if(Y >= tol):
            arry = np.append(arry, count)
        count = count + 1
    return arry
    # <<<<< END YOUR CODE

In [125]:
# Verify your function
print(important_indexes())

[  16.   17.   18.   19.   20.   21.   22.   23.   24.   25.   26.   27.
   28.   29.   30.   31.   48.   49.   50.   51.   52.   53.   54.   55.
   56.   57.   58.   59.   60.   61.   62.   63.   80.   81.   82.   83.
   84.   85.   86.   87.   88.   89.   90.   91.   92.   93.   94.   95.
  112.  113.  114.  115.  116.  117.  118.  119.  120.  121.  122.  123.
  124.  125.  126.  127.  144.  145.  146.  147.  148.  149.  150.  151.
  152.  153.  154.  155.  156.  157.  158.  159.  176.  177.  178.  179.
  180.  181.  182.  183.  184.  185.  186.  187.  188.  189.  190.  191.
  208.  209.  210.  211.  212.  213.  214.  215.  216.  217.  218.  219.
  220.  221.  222.  223.  240.  241.  242.  243.  244.  245.  246.  247.
  248.  249.  250.  251.  252.  253.  254.  255.  272.  273.  274.  275.
  276.  277.  278.  279.  280.  281.  282.  283.  284.  285.  286.  287.
  304.  305.  306.  307.  308.  309.  310.  311.  312.  313.  314.  315.
  316.  317.  318.  319.  336.  337.  338.  339.  3

## Probability of generating data from a Gaussian model (30 P)

Consider a multivariate Gaussian distribution of mean vector `m` and covariance `S`. The probability associated to a vector `x` is given by:

$$
p(\boldsymbol{x};(\boldsymbol{m},S)) = \frac{1}{\sqrt{(2\pi)^d \mathrm{det}(S)}} \exp \Big( - \frac12 (\boldsymbol{x}-\boldsymbol{m})^\top S^{-1} (\boldsymbol{x}-\boldsymbol{m})\Big)
$$

We consider the calculation of the probability of observing a certain dataset 

$$
\mathcal{D} = (\boldsymbol{x}^{(1)},\dots,\boldsymbol{x}^{(N)})
$$

assuming the data is generated according to a Gaussian distribution of fixed parameters $\boldsymbol{m}$ and $S$. Such probability density is given by the formula:

$$
\log P(\mathcal{D};(\boldsymbol{m},S)) = \log \prod_{i=1}^N p(\boldsymbol{x}^{(i)};(\boldsymbol{m},S))
$$

The function below implements such function:

In [105]:
import numpy,numpy.linalg,utils

def logp(X,m,S):
    
    # Find the number of dimensions from the data vector
    d = X.shape[1]
    
    # Invert the covariance matrix
    Sinv = numpy.linalg.inv(S)
    
    # Compute the quadratic terms for all data points
    Q = -0.5*(numpy.dot(X-m,Sinv)*(X-m)).sum(axis=1)
    
    # Raise them quadratic terms to the exponential
    Q = numpy.exp(Q)
    
    # Divide by the terms in the denominator
    P = Q / numpy.sqrt((2*numpy.pi)**d * numpy.linalg.det(S))
    
    # Take the product of the probability of each data points
    Pprod = numpy.prod(P)
    
    # Return the log-probability
    return numpy.log(Pprod)


Evaluation of this function for various datasets and parameters provided in the file `utils.py` gives the following probabilities:

In [106]:
print('%11.4f'%logp(utils.X1,utils.m1,utils.S1))
print('%11.4f'%logp(utils.X2,utils.m2,utils.S2))
print('%11.4f'%logp(utils.X3,utils.m3,utils.S3))

(5, 1)
(1, 3)
(3, 3)
   -13.0068
(20, 1)
(1, 100)
(100, 100)
       -inf
(100, 1)
(1, 450)
(450, 450)
       -inf


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log


This function is numerically instable for multiple reasons. The product of many probabilities, the inversion of a large covariance matrix, and the computation of its determinant, are all potential causes for overflow. Thus, we would like to find a numerically robust way of performing each of these.

* Implement a numerically stable version of the function `logp`
* Evaluate it on the same datasets and parameters as the function `logp`

In [136]:
def logp_robust(X,m,S):
    # >>>>> YOUR CODE HERE
    # Find the number of dimensions from the data vector
    d = X.shape[1]
    
    # Invert the covariance matrix
    Sinv = numpy.linalg.inv(S)
    
    # Compute the quadratic terms for all data points
    Q = -0.5*(numpy.dot(X-m,Sinv)*(X-m)).sum(axis=1)
    
    
    # Raise them quadratic terms to the exponential
    Q = numpy.exp(Q)
    
    # Divide by the terms in the denominator
    P = Q / numpy.sqrt((2*numpy.pi)**d * numpy.linalg.det(S))
    
    print P.shape
    
    # Take the product of the probability of each data points
    Pprod = numpy.prod(P)
    
    print Pprod
    
    # Return the log-probability
    return numpy.log(Pprod)
    # <<<<< END YOUR CODE

In [137]:
# Verify your function
print('%11.4f'%logp_robust(utils.X1,utils.m1,utils.S1))
print('%11.4f'%logp_robust(utils.X2,utils.m2,utils.S2))
print('%11.4f'%logp_robust(utils.X3,utils.m3,utils.S3))

(5,)
2.2450785299946257e-06
   -13.0068
(20,)
0.0
       -inf
(100,)
0.0
       -inf


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
